In [12]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"
# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,027 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,069 kB]
Fetched 4,433 kB in 2s (2,

In [13]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-04-04 19:44:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  6.23MB/s    in 0.2s    

2023-04-04 19:44:16 (6.23 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [15]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok|2015-08-31 00:00:00|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...| 

### Create DataFrames to match tables

In [16]:
#@title Default title text
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
read_df = df.select(["review_id","customer_id","product_id","product_parent","product_title","review_date","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
read_df.show()

+--------------+-----------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+
|     review_id|customer_id|product_id|product_parent|       product_title|        review_date|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+
|R1NBG94582SJE2|    2975964|B00I01JQJM|     860486164|GoPro Rechargeabl...|2015-08-31 00:00:00|          5|            0|          0|   N|                Y|
|R273DCA6Y0H9V7|   23526356|B00TCO0ZAA|     292641483|Professional 58mm...|2015-08-31 00:00:00|          5|            0|          0|   N|                Y|
| RQVOXO7WUOFK6|   52764145|B00B7733E0|      75825744|Spy Tec Z12 Motio...|2015-08-31 00:00:00|          2|            1|          1|   N|                Y|
|R1KWKSF21PO6HO|   47348933|B006ZN4U34|     789352955|Cele

In [17]:
# Create the vine_table. DataFrame
vine_df = read_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1NBG94582SJE2|          5|            0|          0|   N|                Y|
|R273DCA6Y0H9V7|          5|            0|          0|   N|                Y|
| RQVOXO7WUOFK6|          2|            1|          1|   N|                Y|
|R1KWKSF21PO6HO|          5|            0|          0|   N|                Y|
|R38H3UO1J190GI|          5|            1|          1|   N|                Y|
|R3NPIFKLR19NQA|          3|            0|          0|   N|                Y|
|R3MBE6UCH3435E|          3|            8|          8|   N|                N|
|R2E7A4FF0PVY5Q|          5|            0|          1|   N|                Y|
| R3R8JDQ2BF4NM|          5|            0|          2|   N|                Y|
|R1YND4BS823GN5|          1|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

### Deliverable 2 


#1.Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20



In [18]:
#1.Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
from pyspark.sql.functions import col
total_votes_df = vine_df.filter(col("total_votes") >= 20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y|
|R3EM8C9CKA6GPK|          2|           41|        115|   N|                N|
|R29THVJFO35FZA|          5|           48|         50|   N|                Y|
|R3V8FKXIHBLWEL|          5|           91|        108|   N|     

In [19]:
#2. Filter the new DataFrame or table created in Step 1 and create a new DataFrame "helpful_votes"  divided by "total_votes" is equal to or greater than 50%
helpful_votes_df = total_votes_df.filter("helpful_votes / total_votes >= 0.5")
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y|
|R29THVJFO35FZA|          5|           48|         50|   N|                Y|
|R3V8FKXIHBLWEL|          5|           91|        108|   N|                Y|
|R3JIIPQ8F0P7QS|          4|           22|         23|   N|     

In [22]:
#3. Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
paid_df = helpful_votes_df.filter("vine == 'Y'")
paid_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37L6YZDX0P3J9|          1|           28|         35|   Y|                N|
|R2JT4XRQO2KASL|          5|           38|         41|   Y|                N|
|R10XRAOBFFSTL2|          5|           44|         47|   Y|                N|
| R1X8F3R1K1SF5|          2|           30|         31|   Y|                N|
|R326QV66CKBB43|          4|          127|        139|   Y|                N|
| RGXLFTR31J561|          5|           29|         34|   Y|                N|
|R2ZS3WCJYUDQH0|          3|          101|        115|   Y|                N|
|R31771VBPRDVJS|          4|           75|         83|   Y|                N|
| R8O8KM73GO190|          5|           21|         28|   Y|                N|
| RPU1GURA1NOT0|          5|           33|         35|   Y|     

In [23]:
#4. Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
npaid_df = helpful_votes_df.filter("vine == 'N'")
npaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y|
|R29THVJFO35FZA|          5|           48|         50|   N|                Y|
|R3V8FKXIHBLWEL|          5|           91|        108|   N|                Y|
|R3JIIPQ8F0P7QS|          4|           22|         23|   N|     

In [28]:
#5 Review Calculations total number of reviews
total_paid_reviews_df = helpful_votes_df.filter("vine == 'Y'").count()
print(total_paid_reviews_df)

607


In [30]:
#unpaid total reviews
total_unpaid_reviews_df = helpful_votes_df.filter("vine == 'N'").count()
print(total_unpaid_reviews_df)

50522


In [31]:
# 5 star reviews (Paid)
five_star_paid_df = paid_df.filter("star_rating ==5").count()
print(five_star_paid_df)

257


In [32]:
# 5 star reviews (Un-Paid)

five_star_npaid_df = npaid_df.filter("star_rating ==5").count()
print(five_star_npaid_df)

25220


In [33]:
#Percentage of 5-star reviews for the two types of review (paid vs unpaid).
percent_paid_df = five_star_paid_df / total_paid_reviews_df * 100
print(percent_paid_df)


42.33937397034596


In [34]:
#unpaid
percent_unpaid_df = five_star_npaid_df / total_unpaid_reviews_df * 100
print(percent_unpaid_df)

49.91884723486798
